In [1]:
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.optimization import SingleMetricOptimizer, Gingleator
from gerrychain.tree import recursive_seed_part
from functools import partial
import pandas as pd
import json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import random

random.seed(2024)

In [2]:
graph = Graph.from_json("./VA/VA1.json");

In [3]:
print(graph.nodes()[0])

{'COUNTYFP': '001', 'LOCALITY': 'Accomack County', 'VTDST': '000101', 'PRECINCT': 'Chincoteague', 'G20PRED': 837, 'G20PRER': 1618, 'G20USSD': 915, 'G20USSR': 1563, 'TOTPOP': 3345.0, 'HISP': 96.0, 'NH_WHITE': 3035.0, 'NH_BLACK': 28.0, 'NH_AMIN': 8.0, 'NH_ASIAN': 19.0, 'NH_NHPI': 0.0, 'NH_OTHER': 7.0, 'NH_2MORE': 152.0, 'VAP': 2917.0, 'HVAP': 66.0, 'WVAP': 2674.0, 'BVAP': 25.0, 'AMINVAP': 6.0, 'ASIANVAP': 17.0, 'NHPIVAP': 0.0, 'OTHERVAP': 7.0, '2MOREVAP': 122.0, 'G18USSD': 2.111475409836066, 'G18USSR': 10.934426229508198, 'SEND': '20'}


In [4]:
POPCOL = "TOTPOP"
SEN_DISTS = 40
EPS = 5
TOTPOP = sum(graph.nodes()[n][POPCOL] for n in graph.nodes())

chain_updaters = {
"population": updaters.Tally(POPCOL, alias="population"),
"VAP": updaters.Tally("VAP"),
"HVAP": updaters.Tally("HVAP")
}

initial_partition = Partition.from_random_assignment(
    graph=graph,
    n_parts=SEN_DISTS,
    epsilon=EPS,
    pop_col=POPCOL,
    updaters=chain_updaters
)

proposal = partial(
    proposals.recom,
    pop_col=POPCOL,
    pop_target=TOTPOP/SEN_DISTS,
    epsilon=EPS,
    node_repeats=1
)

constraints = constraints.within_percent_of_ideal_population(initial_partition, EPS)

/Users/lintingxuan/miniconda3/envs/gerry/lib/python3.11/site-packages/gerrychain/tree.py:704: BipartitionWarning: 
Failed to find a balanced cut after 1000 attempts.
If possible, consider enabling pair reselection within your
MarkovChain proposal method to allow the algorithm to select
a different pair of districts for recombination.
  warnings.warn(


RuntimeError: Could not find a possible cut after 10000 attempts.

In [ ]:
num_cut_edges = lambda p: len(p["cut_edges"])

optimizer = SingleMetricOptimizer(
    proposal=proposal,
    constraints=constraints,
    initial_state=initial_partition,
    optimization_metric=num_cut_edges,
    maximize=False
)

In [ ]:
total_steps = 10000

# Short Bursts
min_scores_sb = np.zeros(total_steps)
for i, part in enumerate(optimizer.short_bursts(5, 2000, with_progress_bar=True)):
    min_scores_sb[i] = optimizer.best_score

# Simulated Annealing
min_scores_anneal = np.zeros(total_steps)
for i, part in enumerate(
    optimizer.simulated_annealing(
        total_steps,
        optimizer.jumpcycle_beta_function(200, 800),
        beta_magnitude=1,
        with_progress_bar=True
    )
):
    min_scores_anneal[i] = optimizer.best_score

# Tilted Runs
min_scores_tilt = np.zeros(total_steps)
for i, part in enumerate(optimizer.tilted_run(total_steps, p=0.125, with_progress_bar=True)):
    min_scores_tilt[i] = optimizer.best_score

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.plot(min_scores_sb, label="Short Bursts")
plt.plot(min_scores_anneal, label="Simulated Annealing")
plt.plot(min_scores_tilt, label="Tilted Run")
plt.xlabel("Steps", fontsize=20)
plt.ylabel("Min #CutEdges Observered", fontsize=20)
plt.legend()
plt.show()

In [ ]:
gingles = Gingleator(
    proposal,
    constraints,
    initial_partition,
    minority_pop_col="BVAP",
    total_pop_col="TOTPOP",
    score_function=Gingleator.reward_partial_dist
)

In [ ]:
total_steps = 10000

# Short Bursts
max_scores_sb = np.zeros(total_steps)
scores_sb = np.zeros(total_steps)
for i, part in enumerate(gingles.short_bursts(10, 1000, with_progress_bar=True)):
    max_scores_sb[i] = gingles.best_score
    scores_sb[i] = gingles.score(part)

# Simulated Annealing
max_scores_anneal = np.zeros(total_steps)
scores_anneal = np.zeros(total_steps)
for i, part in enumerate(
    gingles.simulated_annealing(
        total_steps,
        gingles.jumpcycle_beta_function(1000, 4000),
        beta_magnitude=500,
        with_progress_bar=True
    )
):
    max_scores_anneal[i] = gingles.best_score
    scores_anneal[i] = gingles.score(part)

# Tilted Runs
max_scores_tilt = np.zeros(total_steps)
scores_tilt = np.zeros(total_steps)
for i, part in enumerate(gingles.tilted_run(total_steps, 0.125, with_progress_bar=True)):
    max_scores_tilt[i] = gingles.best_score
    scores_tilt[i] = gingles.score(part)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.plot(max_scores_sb, label="Short Bursts")
plt.plot(max_scores_anneal, label="Simulated Annealing")
plt.plot(max_scores_tilt, label="Tilted Run")
plt.xlabel("Steps", fontsize=20)
plt.ylabel("Max Score Observered", fontsize=20)
plt.legend()
plt.show()